#Neural Network - Time Series Prediction

####Based off shallow NNet architecture, inputs consist of data lagged 5,10,15 days with predictions consisting of a directional forecast of the index in 5 days time. 

The usage of weekly lagged data reduces issues with the fact that market [returns over the weekend](http://onlinelibrary.wiley.com/doi/10.1111/j.1468-5957.1988.tb00130.x/abstract) are not the same as intra week returns. The notebook is divided up into the following stages:

1. Introduction and Setup
1. Basic Prediction - Price & Volume Data
1. Advanced Prediction - Technical Indicators
1. Further Investigation - Fear & Greed Indicator Implementation

##1. Introduction and Setup

###Get our Time Series Data

In [1]:
#importing libraries and tools
%pylab inline
from yahoo_finance import Share
from pprint import pprint
import pandas as pd
from __future__ import division
import numpy as np
import talib as tal
from scipy import optimize
from sklearn.metrics import accuracy_score
import sklearn.datasets as datasets
from sklearn import cross_validation
import time
import random
import math
import collections

Populating the interactive namespace from numpy and matplotlib


We are using S&P 500 data for this example, we are collecting data from March 2012 to January 2015. We are collecting a larger sample of data called techtraindata for later usage with indicators which need backfilling.

In [2]:
#s&p 500
spx = Share('^GSPC')

#training data (note extended tech dataset for technical indicators)
traindata = spx.get_historical('2012-03-01','2015-01-01') 
techtraindata = spx.get_historical('2011-12-01','2015-01-01')

pprint(traindata[0]) #here is a sample of we have requested

{'Adj_Close': '2058.899902',
 'Close': '2058.899902',
 'Date': '2014-12-31',
 'High': '2085.580078',
 'Low': '2057.939941',
 'Open': '2082.110107',
 'Symbol': '%5eGSPC',
 'Volume': '2606070000'}


In [3]:
print(len(traindata),len(techtraindata)) #this is the difference between our extended time series for the technical indicators

714 775


###Bringing a Neural Network to the party

**This has one hidden layer with 6 neurons**. The number of hidden neurones was chosen by the calculation of **Sqrt(36 + 1) s 6**. It is trained with a Truncated Newton Constrained Algorithm. This code was gathered off stackoverflow (I believe) some time ago, I have been unable to match the code to an author after some searching. I do not take credit for its core construction, however I utilised its core architecture and made some changes off the back of [this](https://github.com/stephencwelch/Neural-Networks-Demystified) course.

In [4]:
class Neural_Network(object):
    
    def __init__(self, reg_lambda=0, epsilon_init=0.12, hidden_layer_size=6, opti_method='TNC', maxiter=500):
        self.reg_lambda = reg_lambda
        self.epsilon_init = epsilon_init
        self.hidden_layer_size = hidden_layer_size
        self.activation_func = self.sigmoid
        self.activation_func_prime = self.sigmoid_prime
        self.method = opti_method
        self.maxiter = maxiter
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_prime(self, z):
        sig = self.sigmoid(z)
        return sig * (1 - sig)
    
    def sumsqr(self, a):
        return np.sum(a ** 2)
    
    def rand_init(self, l_in, l_out):
        return np.random.rand(l_out, l_in + 1) * 2 * self.epsilon_init - self.epsilon_init
    
    def pack_thetas(self, t1, t2):
        return np.concatenate((t1.reshape(-1), t2.reshape(-1)))
    
    def unpack_thetas(self, thetas, input_layer_size, hidden_layer_size, num_labels):
        t1_start = 0
        t1_end = hidden_layer_size * (input_layer_size + 1)
        t1 = thetas[t1_start:t1_end].reshape((hidden_layer_size, input_layer_size + 1))
        t2 = thetas[t1_end:].reshape((num_labels, hidden_layer_size + 1))
        return t1, t2
    
    def _forward(self, X, t1, t2):
        m = X.shape[0]
        ones = None
        if len(X.shape) == 1:
            ones = np.array(1).reshape(1,)
        else:
            ones = np.ones(m).reshape(m,1)
        
        # Input layer
        a1 = np.hstack((ones, X))
        
        # Hidden Layer
        z2 = np.dot(t1, a1.T)
        a2 = self.activation_func(z2)
        a2 = np.hstack((ones, a2.T))
        
        # Output layer
        z3 = np.dot(t2, a2.T)
        a3 = self.activation_func(z3)
          
        return a1, z2, a2, z3, a3
    
    def function(self, thetas, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda):
        t1, t2 = self.unpack_thetas(thetas, input_layer_size, hidden_layer_size, num_labels)
        
        m = X.shape[0]
        Y = np.eye(num_labels)[y]
        
        _, _, _, _, h = self._forward(X, t1, t2)
        costPositive = -Y * np.log(h).T
        costNegative = (1 - Y) * np.log(1 - h).T
        cost = costPositive - costNegative
        J = np.sum(cost) / m
        
        if reg_lambda != 0:
            t1f = t1[:, 1:]
            t2f = t2[:, 1:]
            reg = (self.reg_lambda / (2 * m)) * (self.sumsqr(t1f) + self.sumsqr(t2f))
            J = J + reg
        return J
        
    def function_prime(self, thetas, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda):
        t1, t2 = self.unpack_thetas(thetas, input_layer_size, hidden_layer_size, num_labels)
        
        m = X.shape[0]
        t1f = t1[:, 1:]
        t2f = t2[:, 1:]
        Y = np.eye(num_labels)[y]
        
        Delta1, Delta2 = 0, 0
        for i, row in enumerate(X):
            a1, z2, a2, z3, a3 = self._forward(row, t1, t2)
            
            # Backprop
            d3 = a3 - Y[i, :].T
            d2 = np.dot(t2f.T, d3) * self.activation_func_prime(z2)
            
            Delta2 += np.dot(d3[np.newaxis].T, a2[np.newaxis])
            Delta1 += np.dot(d2[np.newaxis].T, a1[np.newaxis])
            
        Theta1_grad = (1 / m) * Delta1
        Theta2_grad = (1 / m) * Delta2
        
        if reg_lambda != 0:
            Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (reg_lambda / m) * t1f
            Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (reg_lambda / m) * t2f
        
        return self.pack_thetas(Theta1_grad, Theta2_grad)
    
    def fit(self, X, y):
        num_features = X.shape[0]
        input_layer_size = X.shape[1]
        num_labels = len(set(y))
        
        theta1_0 = self.rand_init(input_layer_size, self.hidden_layer_size)
        theta2_0 = self.rand_init(self.hidden_layer_size, num_labels)
        thetas0 = self.pack_thetas(theta1_0, theta2_0)
        
        options = {'maxiter': self.maxiter}
        _res = optimize.minimize(self.function, thetas0, jac=self.function_prime, method=self.method, 
                                 args=(input_layer_size, self.hidden_layer_size, num_labels, X, y, 0), options=options)
        
        self.t1, self.t2 = self.unpack_thetas(_res.x, input_layer_size, self.hidden_layer_size, num_labels)
    
    def predict(self, X):
        return self.predict_proba(X).argmax(0)
    
    def predict_proba(self, X):
        _, _, _, _, h = self._forward(X, self.t1, self.t2)
        return h


##2. Basic Prediction - Price & Volume Data

From our previous data import we are left with a dictionary of data in descending date order (as you could see from the previous pprint output).

Below we go onto extract the high,low and close prices for the S&P 500 along with the volume for each day, we then normalise close and volume data. 

In [5]:
highprice = np.array([row['High'] for row in techtraindata]).astype(np.float)

lowprice = np.array([row['Low'] for row in techtraindata]).astype(np.float)

close = np.array([row['Close'] for row in techtraindata]).astype(np.float)
closeprices = pd.DataFrame(close / (close.max()*1.1))

vol = np.array([row['Volume'] for row in techtraindata]).astype(np.float)
volume = pd.DataFrame(vol / (vol.max()*1.1))

The data is lagged in 5 day intervals and arrangd into an array. We call the unlagged column the forecast - this is so we can (if needed) see what we would be trying to predict from the other four columns. We are naming these columns t, lag5, lag10, lag15 however they are actually when considering the time series dates, lagged by 5 extra days. I deemed this to be a suitably simple method generating the data in a form that was easy to train the NNet with.

In [6]:
#using unlagged value as forecast to make comparison against
prices = pd.concat([closeprices, closeprices.shift(5), closeprices.shift(10), closeprices.shift(15), closeprices.shift(20)], axis=1)
prices.columns = ['forcast','t','lag5','lag10','lag15']

volumes = pd.concat([volume, volume.shift(5), volume.shift(10), volume.shift(15), volume.shift(20)], axis=1)
volumes.columns = ['forecast', 't','lag5','lag10','lag15']

prices = [[float(column) for column in row] for row in prices[prices.lag15.notnull()].values]
volumes = [[float(column) for column in row] for row in volumes[volumes.lag15.notnull()].values]

print(len(prices),len(traindata),len(techtraindata))

755 714 775


We can see we have more data than our training data set, this is because we wanted to ensure when the data was lagged the minimum range (forecast vs max lag) had sufficent rows. In this case 755 is left after we lagged the original data (775) as 755>714 we are okay.

Here we begin to put the data into the arrays that the neural network will recieve as an input and expected output to train from.

In [7]:
X = [] ; y = []

#iterating over our data
for i in range(0,len(traindata)-20): #since we have lagged by 20 days at most
    i = int(i) 
    val = 0
    if prices[i+20][0]>prices[i+20][1]:
        val = 1 #s&p 500 is higher the following week
    else:
        val = 0 #s&p 500 is lower the following week
    
    #combining our data into the training arrays
    X.extend([[prices[i+20][1],prices[i+20][2],prices[i+20][3],prices[i+20][4],volumes[i+20][1],volumes[i+20][2], volumes[i+20][3], volumes[i+20][4]]])
    y.extend([val]) #our predictions
        

X = np.array(X) ; y = np.array(y)

print(X.shape, y.shape) #ensuring we have a match

(694, 8) (694,)


###Training the Model
Before we begin to train the model, we partition our availble data into a training and test set so we can cross validate our predictions. We use a 90:10 split for training and test.

In [8]:
# 9:1 - train:test, initial epsilon=0.12, lambda=0
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
NN = Neural_Network()
NN.fit(X_train, y_train)

###Model Accuracy

In [9]:
accuracy_score(y_test, NN.predict(X_test))

0.61428571428571432

Here we find the model accuracy of **61%** this is fairly low considering this is a fairly simple prediction requirement, however considering the lack of data that the model has it is not aweful.

# 3. Advanced Prediction - Technical Indicators

### Adding technical indicators (with lagged view)
Please see [TA-LIB](https://github.com/mrjbq7/ta-lib) documentation for full details on the functions.

####MACD Indicator Calculation

**Moving Average Convergence / Divergence** is the difference betwen two exponentially smoothed moving averags of closing prices. Here we have used a fast period of 12, a slow period of 26, and a signal period of 9. The period choices were arbitrary as optimisation is needed to "correctly" choose such periods. These are supposedly the most commonly used settings. This is a suitable view to have as market participants will be looking at the same data.

In [10]:
macd, macdsignal, macdhist = tal.MACD(np.fliplr([close])[0], fastperiod=12, slowperiod=26, signalperiod=9)
macd = pd.DataFrame(macd);macdsignal = pd.DataFrame(macdsignal);macdhist = pd.DataFrame(macdhist)

macd = macd[macd.notnull()].iloc[::-1];macdsignal = macdsignal[macdsignal.notnull()].iloc[::-1];macdhist = macdhist[macdhist.notnull()].iloc[::-1]
macddata = pd.concat([macd, macd.shift(5), macd.shift(10), macd.shift(15), macd.shift(20),macdsignal, macdsignal.shift(5), macdsignal.shift(10), macdsignal.shift(15), macdsignal.shift(20), macdhist, macdhist.shift(5), macdhist.shift(10), macdhist.shift(15), macdhist.shift(20)], axis=1).dropna()

macddata.columns = ['forcast','t','lag5','lag10','lag15','forcast','t','lag5','lag10','lag15','forcast','t','lag5','lag10','lag15']
macddata = [[float(column) for column in row] for row in macddata.values]
print(len(macddata),len(traindata),len(techtraindata))

722 714 775


#### RSI Indicator Calculation

**The Relative Strength Index** is an oscillator which gauges the relative strength of the security by considering up days and down days with a smoothing effect. Here a timeperiod of 14 has been chosen as this is the most popular setting.

In [11]:
rsi = pd.DataFrame(tal.RSI(np.fliplr([close])[0], timeperiod=14))

rsi = rsi[rsi.notnull()].iloc[::-1]
rsidata = pd.concat([rsi, rsi.shift(5), rsi.shift(10), rsi.shift(15), rsi.shift(20)], axis=1).dropna()

rsidata.columns = ['forcast','t','lag5','lag10','lag15']
rsidata = [[float(column) for column in row] for row in rsidata.values]
print(len(rsidata),len(traindata),len(techtraindata))

741 714 775


#### ADX Indicator Calculation

**The Average Directional Index** quantifies trend strength based on moving averages, here we have set the most common setting as 14

In [12]:
adx = pd.DataFrame(tal.ADX(np.fliplr([highprice])[0], np.fliplr([lowprice])[0], np.fliplr([close])[0], timeperiod=14))

adx = adx[adx.notnull()].iloc[::-1]
adxdata = pd.concat([adx, adx.shift(5), adx.shift(10), adx.shift(15), adx.shift(20)], axis=1).dropna()

adxdata.columns = ['forcast','t','lag5','lag10','lag15']
adxdata = [[float(column) for column in row] for row in adxdata.values]
print(len(adxdata),len(traindata),len(techtraindata))

728 714 775


#### STOCH Indicator Calculation

**Stochastics** provides an oscillator that determines the close price in reference to recent trading range as an indicator of potential bullish and bearish trends ahead. Again popular settings are chosen.

In [13]:
slowk, slowd = tal.STOCH(np.fliplr([highprice])[0], np.fliplr([lowprice])[0], np.fliplr([close])[0], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
slowk = pd.DataFrame(slowk);slowd = pd.DataFrame(slowd)

slowk = slowk[slowk.notnull()].iloc[::-1];slowd = slowd[slowd.notnull()].iloc[::-1]
stochdata = pd.concat([slowk, slowk.shift(5), slowk.shift(10), slowk.shift(15), slowk.shift(20), slowd, slowd.shift(5), slowd.shift(10), slowd.shift(15), slowd.shift(20)], axis=1).dropna()

stochdata.columns = ['forcast','t','lag5','lag10','lag15','forcast','t','lag5','lag10','lag15']
stochdata = [[float(column) for column in row] for row in stochdata.values]
print(len(stochdata),len(traindata),len(techtraindata))

738 714 775


In [14]:
#put our data into an array ready for the neural network:

X = [] #creating a list
y = []

for i in range(0,len(traindata)-20):
    i = int(i)
    val = 0
    if prices[i+20][0]>prices[i+20][1]:
        val = 1
    else:
        val = 0
    
    X.extend([[prices[i+20][1],prices[i+20][2],prices[i+20][3],prices[i+20][4],volumes[i+20][1],volumes[i+20][2], volumes[i+20][3], volumes[i+20][4], macddata[i+20][1],macddata[i+20][2],macddata[i+20][3],macddata[i+20][4], macddata[i+20][6],macddata[i+20][7],macddata[i+20][8],macddata[i+20][9],macddata[i+20][11],macddata[i+20][12],macddata[i+20][13],macddata[i+20][14],rsidata[i+20][1],rsidata[i+20][2],rsidata[i+20][3],rsidata[i+20][4],adxdata[i+20][1],adxdata[i+20][2],adxdata[i+20][3],adxdata[i+20][4],stochdata[i+20][1],stochdata[i+20][2],stochdata[i+20][3],stochdata[i+20][4],stochdata[i+20][6],stochdata[i+20][7],stochdata[i+20][8],stochdata[i+20][9]]])
    y.extend([val])
        

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

(694, 36) (694,)


In [15]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
NN = Neural_Network()
NN.fit(X_train, y_train)

accuracy_score(y_test, NN.predict(X_test))

0.91428571428571426

Here we see a vastly improved accuracy of **91%** as compared to the model trained on only price and volume data.

### 4. Further Investigation - Fear & Greed Indicator Implementation

#### Adding a Fear & Greed Index

We can look to other non price data such as the [Fear & Greed index](http://money.cnn.com/data/fear-and-greed/) that can give our model more predictive power using data that does not originate from internal price action.

Incidently I have performed some Granger Causality analysis on this index, looking at its predictive power on the S&P500 index, it concluded that the index changes **did not** Granger-Cause the S&P 500 returns. However this must be revisited with more sophisticated techniques to remove seasonality and various factors affecting the tests.

In [16]:
#Import our data 
with open('FGDATA.csv', 'r') as f:
    data = [i.split(",") for i in f.read().split()]

#we have data from 1st febuary 2014 to 1st jan 2015
indexdata = np.array([row[1] for row in data]).astype(np.float)
indexdata = np.fliplr([indexdata])[0]

In [17]:
indexdata = pd.DataFrame(indexdata)
index = pd.concat([indexdata, indexdata.shift(5), indexdata.shift(10), indexdata.shift(15), indexdata.shift(20)], axis=1)
index.columns = ['forcast','t','lag5','lag10','lag15']
index = [[float(column) for column in row] for row in index[index.lag15.notnull()].values]

print(len(index),len(traindata),len(techtraindata))

717 714 775


In [18]:
#put our data into an array ready for the neural network:

X = [] #creating a list
y = []

for i in range(0,len(traindata)-20):
    i = int(i)
    val = 0
    if prices[i+20][0]>prices[i+20][1]:
        val = 1
    else:
        val = 0
        
    X.extend([[prices[i+20][1],prices[i+20][2],prices[i+20][3],prices[i+20][4],volumes[i+20][1],volumes[i+20][2], volumes[i+20][3], volumes[i+20][4], index[i+20][1],index[i+20][2],index[i+20][3],index[i+20][4]]])
    y.extend([val])
        
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

(694, 12) (694,)


In [19]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
NN = Neural_Network()
NN.fit(X_train, y_train)

accuracy_score(y_test, NN.predict(X_test))

0.74285714285714288

Here we see an inferior prediction accuracy of **74%** compared to the previous model, it is able to outperform the inital model with only price and volume but it does not have a superior predictive power to the model with technical indicators. It appears that despite the F&G index being a useful gauge of sentiment, it does not influence the directional move in the following week as much as technical indicators. This could be due to:

* More people are probably looking to the common technical indicators rather than this bespoke index. 
* Market moves in my belief are self fufilling (unless large fundamental changes occur) people guage what others may make of data and act accordingly to cause their expectation unknowingly.
* Diverse indicators are good at giving a overview of the market however specific changes on a weekly basis are less likely to have correlations to a general status indicator changes

So overall we find that a neural network with one hidden layer with 6 neurons, trained with 36 inputs, consisting of lagged price data with indicators has **91% accuracy** at predicting the directional move of the S&P 500 in the following week.

__I am looking to combine this model with an algorithm running multiple strategies to establish a directional bias in momentum trades.__